In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()


In [ ]:
# Summarise the data by UK Cardholder Spending All Quarterly --------------- Cardholders' Number Total Quarterly ---- TEST mcg = 'All'

UK_spending_by_mcg_All = '''SELECT time_period_value, cardholders, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcg = 'All' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
 
GROUP BY cardholders, destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_mcg_All = bq.read_bq_table_sql(client, UK_spending_by_mcg_All)
df_by_mcg_All.head()

# Caculate UK Domestic Total Spending Quarterly

# Assuming df_by_mcg_All is the DataFrame returned from the BigQuery query
# Then group by 'time_period_value' and sum the 'spend' for each quarter

# Check if df_by_mcg_All is not None and has the expected columns
if df_by_mcg_All is not None and 'time_period_value' in df_by_mcg_All.columns and 'spend' in df_by_mcg_All.columns:
    # Group by quarter and sum the spend
    UK_spending_by_mcg_All = df_by_mcg_All.groupby('time_period_value')['cardholders'].sum().reset_index()
   
 # Rename the column
    UK_spending_by_mcg_All = UK_spending_by_mcg_All.rename(columns={'cardholders': 'Spend_mcg_All_cardholders'})
    print(UK_spending_by_mcg_All)
else:
    print("DataFrame is empty or missing required columns.")
    
    # Save the result to a CSV file
csv_filename = "UK_spending_by_mcg_All.csv"
UK_spending_by_mcg_All.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created successfully.")


In [ ]:
#Top 10 MCG F2F

Top10_MCGs = '''
WITH mcg_quarterly_spend AS (
  SELECT 
    SUM(spend) AS total_spend,
    time_period_value,
    mcg
  FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
  WHERE time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
  AND mcg != 'All'
  AND cardholder_origin_country = 'All' 
  AND cardholder_origin = 'UNITED KINGDOM' 
  AND merchant_channel = 'Face to Face'
    GROUP BY time_period_value, mcg
),
ranked_mcg AS (
  SELECT 
    *,
    RANK() OVER (PARTITION BY time_period_value ORDER BY total_spend DESC) AS mcg_rank
  FROM mcg_quarterly_spend
)
SELECT 
  time_period_value,
  mcg,
  total_spend
FROM ranked_mcg
WHERE mcg_rank <= 10
ORDER BY time_period_value, mcg_rank;
'''

# Run the query and load into a DataFrame
df_Top10_MCGs = client.query(Top10_MCGs).to_dataframe()

# Save to CSV
df_Top10_MCGs.to_csv('Top10_MCGs.csv', index=False)

print("Top 10 MCGs by quarter saved to 'Top10_MCGs.csv'")

In [ ]:
#Last 5 MCG F2F

Last5_MCGs = '''
WITH mcg_last5_spend AS (
  SELECT 
    SUM(spend) AS total_spend,
    time_period_value,
    mcg
  FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
  WHERE time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
  AND mcg != 'All'
  AND cardholder_origin_country = 'All' 
  AND cardholder_origin = 'UNITED KINGDOM' 
  AND merchant_channel = 'Face to Face'
    GROUP BY time_period_value, mcg
),
ranked_mcg AS (
  SELECT 
    *,
    RANK() OVER (PARTITION BY time_period_value ORDER BY total_spend DESC) AS mcg_rank
  FROM mcg_last5_spend
)
SELECT 
  time_period_value,
  mcg,
  total_spend
FROM ranked_mcg
WHERE mcg_rank > (SELECT MAX(mcg_rank) - 5 FROM ranked_mcg)
ORDER BY time_period_value, mcg_rank;
'''

# Run the query and load into a DataFrame
df_Last5_MCGs = client.query(Last5_MCGs).to_dataframe()

# Save to CSV
df_Last5_MCGs.to_csv('Last5_MCGs.csv', index=False)

print("Last 5 MCGs by quarter saved to 'Last5_MCGs.csv'")

In [ ]:
# Line Chart for 2019Q1 as base value = 100 Top10 MCG

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
df = pd.read_csv("Top10_MCGs.csv")

# Pivot the data to have time_period_value as index and mcc as columns
pivot_df = df.pivot(index="time_period_value", columns="mcg", values="total_spend")

# Sort the index to ensure chronological order
pivot_df = pivot_df.sort_index()

# Normalize the data to 2019Q1 = 100
normalized_df = pivot_df.divide(pivot_df.loc["2019Q1"]).multiply(100)

# Filter for quarters from 2019Q1 to 2025Q1
quarters = pd.date_range(start="2019Q1", end="2025Q1", freq="Q").to_period("Q").astype(str)
normalized_df = normalized_df.loc[normalized_df.index.isin(quarters)]

# Plot the line chart
plt.figure(figsize=(12, 6))
for column in normalized_df.columns:
    plt.plot(normalized_df.index, normalized_df[column], label=column)

plt.title("Indexed Total Face to Face Spending by MCG (2019Q1 = 100)")
plt.xlabel("Quarter")
plt.ylabel("Indexed Total Spend")
plt.xticks(rotation=45)
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
#Top 10 MCG Online

Top10_MCGs_Online = '''
WITH mcg_quarterly_spend AS (
  SELECT 
    SUM(spend) AS total_spend,
    time_period_value,
    mcg
  FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
  WHERE time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
  AND mcg != 'All'
  AND cardholder_origin_country = 'All' 
  AND cardholder_origin = 'UNITED KINGDOM' 
  AND merchant_channel = 'Online'
    GROUP BY time_period_value, mcg
),
ranked_mcg AS (
  SELECT 
    *,
    RANK() OVER (PARTITION BY time_period_value ORDER BY total_spend DESC) AS mcg_rank
  FROM mcg_quarterly_spend
)
SELECT 
  time_period_value,
  mcg,
  total_spend
FROM ranked_mcg
WHERE mcg_rank <= 10
ORDER BY time_period_value, mcg_rank;
'''

# Run the query and load into a DataFrame
df_Top10_MCGs_Online = client.query(Top10_MCGs_Online).to_dataframe()

# Save to CSV
df_Top10_MCGs_Online.to_csv('Top10_MCGs_Online.csv', index=False)

print("Top 10 MCGs Online by quarter saved to 'Top10_MCGs_Online.csv'")

In [ ]:
#Last 5 MCG Online

Last5_MCGs_Online = '''
WITH mcg_last5_spend AS (
  SELECT 
    SUM(spend) AS total_spend,
    time_period_value,
    mcg
  FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
  WHERE time_period_value IN (
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2024Q1', '2024Q2', '2024Q3', '2024Q4', '2025Q1'
  )
  AND mcg != 'All'
  AND cardholder_origin_country = 'All' 
  AND cardholder_origin = 'UNITED KINGDOM' 
  AND merchant_channel = 'Online'
    GROUP BY time_period_value, mcg
),
ranked_mcg AS (
  SELECT 
    *,
    RANK() OVER (PARTITION BY time_period_value ORDER BY total_spend DESC) AS mcg_rank
  FROM mcg_last5_spend
)
SELECT 
  time_period_value,
  mcg,
  total_spend
FROM ranked_mcg
WHERE mcg_rank > (SELECT MAX(mcg_rank) - 5 FROM ranked_mcg)
ORDER BY time_period_value, mcg_rank;
'''

# Run the query and load into a DataFrame
df_Last5_MCGs_Online = client.query(Last5_MCGs_Online).to_dataframe()

# Save to CSV
df_Last5_MCGs_Online.to_csv('Last5_MCGs_Online.csv', index=False)

print("Last 5 MCGs Online by quarter saved to 'Last5_MCGs_Online.csv'")

In [ ]:
# Line Chart for 2019Q1 as base value = 100 Top10 Online MCG

import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
df = pd.read_csv("Top10_MCGs_Online.csv")

# Pivot the data to have time_period_value as index and mcc as columns
pivot_df = df.pivot(index="time_period_value", columns="mcg", values="total_spend")

# Sort the index to ensure chronological order
pivot_df = pivot_df.sort_index()

# Normalize the data to 2019Q1 = 100
normalized_df = pivot_df.divide(pivot_df.loc["2019Q1"]).multiply(100)

# Filter for quarters from 2019Q1 to 2025Q1
quarters = pd.date_range(start="2019Q1", end="2025Q1", freq="Q").to_period("Q").astype(str)
normalized_df = normalized_df.loc[normalized_df.index.isin(quarters)]

# Plot the line chart
plt.figure(figsize=(12, 6))
for column in normalized_df.columns:
    plt.plot(normalized_df.index, normalized_df[column], label=column)

plt.title("Indexed Total Online Spending by MCG (2019Q1 = 100)")
plt.xlabel("Quarter")
plt.ylabel("Indexed Total Spend")
plt.xticks(rotation=45)
plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.grid(True)
plt.show()


In [ ]:
#Traceable Graph Top10 Online MCG

import pandas as pd
import plotly.express as px

# Load the CSV file
df = pd.read_csv("Top10_MCGs_Online.csv")

# Pivot the data to have time_period_value as index and mcg as columns
pivot_df = df.pivot(index="time_period_value", columns="mcg", values="total_spend")

# Sort the index to ensure chronological order
pivot_df = pivot_df.sort_index()

# Normalize the data to 2019Q1 = 100
normalized_df = pivot_df.divide(pivot_df.loc["2019Q1"]).multiply(100)

# Define the list of quarters from 2019Q1 to 2025Q1
quarters = [
    '2019Q1', '2019Q2', '2019Q3', '2019Q4',
    '2020Q1', '2020Q2', '2020Q3', '2020Q4',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4',
    '2022Q1', '2022Q2', '2022Q3', '2022Q4',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4',
    '2024Q1', '2024Q2', '2024Q3', '2024Q4',
    '2025Q1'
]
normalized_df = normalized_df.loc[normalized_df.index.isin(quarters)]

# Reshape the DataFrame for Plotly
normalized_df = normalized_df.reset_index().melt(id_vars="time_period_value", var_name="MCG", value_name="Indexed Spend")

# Create interactive chart
fig = px.line(normalized_df, x="time_period_value", y="Indexed Spend", color="MCG",
              title="Indexed Total Online Spending by MCG (2019Q1 = 100)",
              labels={"time_period_value": "Quarter"},
              markers=True)

# Adjust layout for better visibility
fig.update_layout(
    width=1200,
    height=600,
    margin=dict(l=80, r=350, t=60, b=60),
    xaxis_tickangle=-45,
    hovermode="x unified",
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.05,
        font=dict(size=10),
        traceorder="normal",
        itemsizing="constant"
    )
)

fig.show()



In [ ]:
# Growth Rate 2025Q1 to 2019Q1 for Top10_MCGs.csv

import pandas as pd

# Load the CSV file
file_path = "Top10_MCGs.csv"
df = pd.read_csv(file_path)

# Filter for 2019Q1 and 2025Q1
df_filtered = df[df['time_period_value'].isin(['2019Q1', '2025Q1'])]

# Pivot the data to have MCGs as rows and time periods as columns
pivot_df = df_filtered.pivot(index='mcg', columns='time_period_value', values='total_spend')

# Drop rows with missing values in either 2019Q1 or 2025Q1
pivot_df = pivot_df.dropna(subset=['2019Q1', '2025Q1'])

# Calculate growth rate
pivot_df['growth_percent'] = ((pivot_df['2025Q1'] - pivot_df['2019Q1']) / pivot_df['2019Q1']) * 100

# Find the MCG with the highest growth
max_growth_mcg = pivot_df['growth_percent'].idxmax()
max_growth_value = pivot_df['growth_percent'].max()

print(f"The MCG category with the highest growth from 2019Q1 to 2025Q1 is '{max_growth_mcg}' with a growth rate of {max_growth_value:.2f}%.")

In [ ]:
# Growth Rate 2025Q1 to 2019Q1 for Top10_MCGs_Online.csv

import pandas as pd

# Load the CSV file
file_path = "Top10_MCGs_Online.csv"
df = pd.read_csv(file_path)

# Filter for 2019Q1 and 2025Q1
df_filtered = df[df['time_period_value'].isin(['2019Q1', '2025Q1'])]

# Pivot the data to have MCGs as rows and time periods as columns
pivot_df = df_filtered.pivot(index='mcg', columns='time_period_value', values='total_spend')

# Drop rows with missing values in either 2019Q1 or 2025Q1
pivot_df = pivot_df.dropna(subset=['2019Q1', '2025Q1'])

# Calculate growth rate
pivot_df['growth_percent'] = ((pivot_df['2025Q1'] - pivot_df['2019Q1']) / pivot_df['2019Q1']) * 100

# Find the MCG with the highest growth
max_growth_mcg = pivot_df['growth_percent'].idxmax()
max_growth_value = pivot_df['growth_percent'].max()

print(f"The MCG category with the highest growth from 2019Q1 to 2025Q1 is '{max_growth_mcg}' with a growth rate of {max_growth_value:.2f}%.")

In [ ]:
# Identify the MCG Face to Face with the lowest growth in Last5_MCGs.csv

import pandas as pd

# Load the CSV file
file_path = "Last5_MCGs.csv"
df = pd.read_csv(file_path)

# Pivot the data to have MCGs as rows and quarters as columns
pivot_df = df.pivot(index="mcg", columns="time_period_value", values="total_spend")

# Ensure both 2019Q1 and 2025Q1 are present
if "2019Q1" in pivot_df.columns and "2025Q1" in pivot_df.columns:
    # Calculate growth percentage
    pivot_df["growth_percent"] = ((pivot_df["2025Q1"] - pivot_df["2019Q1"]) / pivot_df["2019Q1"]) * 100

    # Find the MCG with the least growth
    least_growth_mcg = pivot_df["growth_percent"].idxmin()
    least_growth_value = pivot_df["growth_percent"].min()

    print(f"The MCG category with the least growth from 2019Q1 to 2025Q1 is '{least_growth_mcg}' with a growth of {least_growth_value:.2f}%.")
else:
    print("Required quarters (2019Q1 and 2025Q1) are not present in the dataset.")


In [ ]:
# Identify the MCG Online with the lowest growth in Last5_MCGs_Online.csv

import pandas as pd

# Load the CSV file
file_path = "Last5_MCGs_Online.csv"
df = pd.read_csv(file_path)

# Pivot the data to have MCGs as rows and quarters as columns
pivot_df = df.pivot(index="mcg", columns="time_period_value", values="total_spend")

# Ensure both 2019Q1 and 2025Q1 are present
if "2019Q1" in pivot_df.columns and "2025Q1" in pivot_df.columns:
    # Calculate growth percentage
    pivot_df["growth_percent"] = ((pivot_df["2025Q1"] - pivot_df["2019Q1"]) / pivot_df["2019Q1"]) * 100

    # Find the MCG with the least growth
    least_growth_mcg = pivot_df["growth_percent"].idxmin()
    least_growth_value = pivot_df["growth_percent"].min()

    print(f"The MCG category with the least growth from 2019Q1 to 2025Q1 is '{least_growth_mcg}' with a growth of {least_growth_value:.2f}%.")
else:
    print("Required quarters (2019Q1 and 2025Q1) are not present in the dataset.")
